<a href="https://colab.research.google.com/github/dvvp/Genasys-300X-Data-Analysis/blob/main/Genasys_Test_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import xlrd

In [21]:
# ignores ids, and gets unique extensions
lst = []
dir = "/content/test_files/"
for file in os.listdir(dir):
  if file.endswith(".xlsx"):
    lst.append(file.split("] ")[1])

wo_id = np.unique(lst).tolist()
wo_id

# filters extensions for passed tests and tests that aren't SNR
wo_id_filtered = list(filter(lambda test : test[-11:] == "PASSED.xlsx" and test[:3] != "SNR", wo_id))
wo_id_filtered

['Maximum Output (Wide-Off, Ch1) - PASSED.xlsx',
 'Maximum Output (Wide-Off, Ch2) - PASSED.xlsx',
 'Sensitivity (MP3 Input, Sound Projection Narrow, VB OFF, (Ch1) - PASSED.xlsx',
 'Sensitivity (MP3 Input, Sound Projection Narrow, VB ON, (Ch1) - PASSED.xlsx',
 'Sensitivity (MP3 Input, Sound Projection Wide, VB OFF, Ch1) - PASSED.xlsx',
 'Sensitivity (MP3 Input, Sound Projection Wide, VB OFF, Ch2) - PASSED.xlsx',
 'Sensitivity (MP3 Input, Sound Projection Wide, VB ON, Ch1) - PASSED.xlsx']

In [ ]:
sheet_names = {'Maximum Output (Wide-Off, Ch1) - PASSED.xlsx' : "max_output_ch1",
 'Maximum Output (Wide-Off, Ch2) - PASSED.xlsx' : "max_output_ch2",
 'Sensitivity (MP3 Input, Sound Projection Narrow, VB OFF, (Ch1) - PASSED.xlsx' : "sens_mp3_narrow_vb_off_ch1",
 'Sensitivity (MP3 Input, Sound Projection Narrow, VB ON, (Ch1) - PASSED.xlsx' : "sens_mp3_narrow_vb_on_ch1",
 'Sensitivity (MP3 Input, Sound Projection Wide, VB OFF, Ch1) - PASSED.xlsx' : "sens_mp3_wide_vb_off_ch1",
 'Sensitivity (MP3 Input, Sound Projection Wide, VB OFF, Ch2) - PASSED.xlsx' : "sens_mp3_wide_vb_off_ch2",
 'Sensitivity (MP3 Input, Sound Projection Wide, VB ON, Ch1) - PASSED.xlsx' : "sens_mp3_wide_vb_on_ch1"}

In [28]:
writer = pd.ExcelWriter('output.xlsx')

for id in wo_id_filtered:
  created_final_df = False
  for file in os.listdir(dir):
    if file.endswith(".xlsx"):
      if (id in file):
        if ("Sensitivity" in id):
          sheet = "RMS Level"
        else:
          sheet = "FFT Spectrum"
        df = pd.ExcelFile(dir + file).parse(sheet)
        df = df.tail(df.shape[0] - 3)
        df.rename(columns = {sheet : "X", "Unnamed: 1" : "Y"}, inplace = True)
        df = df.set_index("X")
        if not created_final_df:
          final_df = pd.DataFrame(df.index)
          final_df = final_df.set_index("X")
          created_final_df = True
        final_df = pd.concat([final_df, df], axis=1)

  max = final_df.max(axis = 1)
  min = final_df.min(axis = 1)
  final_df = final_df.assign(MAX = max, MIN = min)
  final_df = final_df.get(["MAX", "MIN"])
  final_df.to_excel(writer, sheet_name = sheet_names[id])

writer.save()

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


KeyboardInterrupt: ignored